In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from scipy.stats import spearmanr, kendalltau

In [2]:
x_train = pd.read_csv('data/x_train.csv')
y_train = pd.read_csv('data/y_train.csv')

/home/maxima1ist/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
y_train_arr = []

for value in y_train['hit']:
    if isinstance(value, str) and value.isdigit():
        y_train_arr.append(int(value))
    else:
        y_train_arr.append(0)

In [4]:
x_test = pd.read_csv('data/x_test.csv')
y_test = pd.read_csv('data/y_test.csv')

In [5]:
y_test_arr = []

for value in y_test['hit']:
    if isinstance(value, str) and value.isdigit():
        y_test_arr.append(int(value))
    else:
        y_test_arr.append(0)

In [6]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(x_train)
enc.categories_

[array([    15,     16,     23, ..., 224482, 224539, 224542]),
 array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167

In [ ]:
model = LogisticRegression(solver='liblinear', C=10, n_jobs=-1)
model.fit(enc.transform(x_train), y_train_arr)

In [ ]:
y_test_predict_proba = [probas[0] for probas in model.predict_proba(enc.transform(x_test))]

In [ ]:
model.predict_proba(enc.transform(x_test))

In [ ]:
y_test_predict = model.predict(enc.transform(x_test))

In [ ]:
valid_train_results = {}
with open('data/valid_train_results.json') as fout:
    valid_train_results = json.load(fout)

In [ ]:
valid_test_results = {}
with open('data/valid_test_results.json') as fout:
    valid_test_results = json.load(fout)

In [ ]:
list_of_real_tournament_range = []
list_of_predicted_tournament_range = []

curr_i = 0
for tournament_id in valid_test_results:
  tournament = valid_test_results[tournament_id]

  real_tournament_range = []
  predicted_tournament_range = []
  for team in tournament:
    real_mask = team['mask']
    real_count = 0
    for i in range(len(real_mask)):
      if isinstance(real_mask[i], str) and real_mask[i].isdigit():
        real_count += int(real_mask[i])

    predict_proba = [1 for _ in range(len(real_mask))]
    for teammate in team['teamMembers']:
        for question_id in range(len(real_mask)):
            predict_proba[question_id] *= y_test_predict_proba[curr_i]
            curr_i += 1
    predict_proba = [(1 - p) for p in predict_proba]

    real_tournament_range.append((team['team']['id'], real_count / len(real_mask)))
    predicted_tournament_range.append((team['team']['id'], np.prod(predict_proba)))
    
  list_of_real_tournament_range.append(real_tournament_range)
  list_of_predicted_tournament_range.append(predicted_tournament_range)

In [ ]:
spearmanr_coefs = []
kendalltau_coefs = []

for i in range(len(list_of_real_tournament_range)):
  predict_tournament_rating = list_of_predicted_tournament_range[i]
  predict_tournament_rating.sort(key=lambda x:x[1])
  predict_tournament_rating = [pair[0] for pair in predict_tournament_rating]
  
  real_tournament_rating = list_of_real_tournament_range[i]
  real_tournament_rating = [pair[0] for pair in real_tournament_rating]
  coef, p = spearmanr(real_tournament_rating, predict_tournament_rating)
  if coef is not np.nan:
    spearmanr_coefs.append(abs(coef))
  coef, p = kendalltau(real_tournament_rating, predict_tournament_rating)
  if coef is not np.nan:
    kendalltau_coefs.append(abs(coef))